In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import string
from bert_serving.client import BertClient
bc = BertClient()
data=pd.read_csv("/Users/prakash/Desktop/natural-to-sql/Final_Processed/Augmented.csv")
X_train, X_test, y_train, y_test = train_test_split(data["Natural"],data["SQL"], test_size=0.2, random_state=42)

In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prakash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
arr=["AffiliationID_Place_Affiliation.csv","AuthID_AffiliationID.csv","AuthID_FieldID.csv","AuthID_Name.csv"
     ,"ConfID_FieldID.csv","ConfID_PaperID.csv","ConfID_Venue_Year.csv","FieldID_Topic.csv","KeywordID_PaperID.csv"
     ,"PaperID_AuthID.csv","PaperID_FieldID.csv","PaperID_Summary.csv","PaperID_Title.csv"]
maping={}
table={}
for i in range(13):
    data=pd.read_csv("/Users/prakash/Desktop/natural-to-sql/Dataset/"+arr[i])
    table[arr[i]]=data.columns
    for val in data.columns:
        if(val in maping):
            maping[val]=list(set(data[val]).intersection(set(maping[val])))
        else:
            maping[val]=list(set(data[val]))
    maping['num']=[str(i) for i in range(500)]

In [4]:
from difflib import get_close_matches 
import re
r=0
embed=[]
place_holder=[]
match_all=[]
ind_all=[]
y_train_rep=[]
for i in X_train.keys():
#     like=0
    if(X_train[i][-1]=='.' or X_train[i][-1]=='?'):
        arr=X_train[i][:-1].strip().split(" ")
    else:
        arr=X_train[i].strip().split(" ")
#     print(y_train[i])
    if(y_train[i][-1]==';'):
        arr1=re.split('\s|=',y_train[i][:-1].strip())
    else:
        arr1=re.split('\s|=',y_train[i].strip())
#     print(arr1)
    temp=[]
    for j in range(len(arr)):
        if(arr[j] not in stop):
            temp.append(arr[j])
    temp3=[]
    y_train_repl= y_train[i]
    for j in range(len(arr1)):
#         if(arr1[j].lower()=="like"):
#             like=1
        if(arr1[j].lower() not in ["select","where","from","having","order","by","group","","like"]):
            ele=arr1[j]
            if(ele[-1]=="'" and ele[0]=="'"):
                ele_t=ele[1:-1]
            elif ele[-1]=="'":
                ele_t=ele[:-1]
            elif ele[0]=="'":
                ele_t=ele[1:]
            else:
                ele_t=ele
            temp3.append(ele_t)
#     like_ele="ksdbfkjbdsjkbfkjdsb"
#     if(like==1):
#         for ele in temp3:
#             if(ele[-1]=="%" and ele[0]=="%"):
#                 like_ele=ele[1:-1]
#             elif ele[-1]=="%":
#                 like_ele=ele[:-1]
#             elif ele[0]=="%":
#                 like_ele=ele[1:]
#     print(temp3)
#     close=[]
    print(i,r)
    temp1=[]
    for val in temp:
        y=get_close_matches(val, [ele for ele in maping])
#         if(len(y)>0):
#             close.extend(y)
#             temp1.append("$col$")
#         else:
        temp1.append(val)
    match=[]
    temp2=[]
    for val in temp1:
        temp2.append(val.lower())
        for ele in maping:
            if(val in maping[ele]):
                match.append(val)
                temp2.pop()
                temp2.append("$val$")
#                 if(ele not in close):
#                     close.append(ele)
    ind_mat=[]
    for ele in temp3:
        temp8=-1
        for i in range(len(match)):
            if(match[i]==ele):
                temp8=i
                if(ele[0]=="*"):
                    y_train_repl=re.sub('\*'+ele[1:], r'~',y_train_repl)
                else:
                    y_train_repl=re.sub(ele, r'~',y_train_repl)
                break
        ind_mat.append(temp8)
    print(y_train_repl)
#     print(temp2) 
#     print(close,match) 
#     for val in table:
    r+=1
    gio=bc.encode([" ".join(temp2)])
    place_holder.append(temp2)
    print(len(gio.flatten()))
    embed.append(gio.flatten())
    match_all.append(match)
    ind_all.append(ind_mat)
    y_train_rep.append(y_train_repl)
#     if(r>10):
#         break

print(len(embed),len(place_holder))

#Contains begin and ends latter -> problem

266 0
SELECT COUNT(AuthID) FROM PaperID_AuthID WHERE PaperID= '~' 
768
148 1
SELECT AuthID FROM PaperID_AuthID WHERE PaperID= '~' 
768
381 2
SELECT FieldID FROM PaperID_FieldID WHERE PaperID = '~'
768
755 3
SELECT Affiliate,Place FROM AffiliationID_Place_Affiliation WHERE AffiliationID == '~'
768
1187 4
SELECT COUNT(*) FROM AuthID_Name WHERE Name LIKE '%Gam%';
768
1316 5
SELECT PaperID FROM ConfID_PaperID WHERE ConfID='~
768
306 6
SELECT PaperID FROM PaperID_FieldID WHERE FieldID = '~' 
768
192 7
SELECT COUNT(AuthID) FROM PaperID_AuthID WHERE PaperID= '~' 
768
309 8
SELECT COUNT(PaperID) FROM PaperID_FieldID WHERE FieldID = '~' 
768
610 9
SELECT PaperID FROM PaperID_Title WHERE Title LIKE 'From indexing the biomed%'
768
63 10
SELECT PaperID FROM PaperID_AuthID WHERE AuthID = '~'
768
843 11
SELECT Count(AuthID) FROM AuthID_AffiliationID WHERE AffiliationID == '~'
768
1663 12
SELECT PaperID FROM KeywordID_PaperID WHERE KeywordID=~;
768
1068 13
SELECT AuthID FROM AuthID_Name WHERE Name='G

768
1402 105
SELECT FieldID FROM ConfID_FieldID WHERE ConfID='~' ;
768
892 106
SELECT AuthID,AffiliationID FROM AuthID_AffiliationID WHERE AuthID == '~'
768
802 107
SELECT Place From AffiliationID_Place_Affiliation WHERE Affiliate LIKE 'TIS Inc
768
1394 108
SELECT FieldID FROM ConfID_FieldID WHERE ConfID='~' ;
768
310 109
SELECT COUNT(PaperID) FROM PaperID_FieldID WHERE FieldID = '~' 
768
1667 110
SELECT PaperID FROM KeywordID_PaperID WHERE KeywordID=~;
768
1105 111
SELECT AuthID FROM AuthID_Name WHERE Name LIKE 'Cross, %';
768
543 112
SELECT Title FROM PaperID_Title WHERE PaperID = '~'
768
1101 113
SELECT AuthID FROM AuthID_Name WHERE Name='Eddy,';
768
1277 114
SELECT ConfID FROM ConfID_PaperID WHERE PaperID='~';
768
585 115
SELECT PaperID FROM PaperID_Title WHERE Title = 'The Role Of Initiative In Tutorial Dialogue'
768
1233 116
SELECT COUNT(*) FROM ConfID_PaperID WHERE ConfID='~' ;
768
478 117
SELECT Summary from PaperID_Summary WHERE PaperID = '~' 
768
1420 118
SELECT FieldID FROM 

768
528 210
SELECT Title FROM PaperID_Title WHERE PaperID = '~'
768
346 211
SELECT FieldID FROM PaperID_FieldID WHERE PaperID = '~'
768
1156 212
SELECT COUNT(*) FROM AuthID_Name WHERE Name LIKE 'Ya%';
768
1635 213
SELECT Topic FROM FieldID_Topic WHERE FieldID=~;
768
674 214
SELECT Affiliate FROM AffiliationID_Place_Affiliation WHERE Place LIKE '%La Tronche France%'
768
808 215
SELECT Place From AffiliationID_Place_Affiliation WHERE Affiliate LIKE 'Financial Crimes Enforcement Network
768
724 216
SELECT Place,Affiliate,AffiliationID FROM AffiliationID_Place_Affiliation WHERE Affiliate LIKE '%Office%'
768
780 217
SELECT Affiliate,Place FROM AffiliationID_Place_Affiliation WHERE AffiliationID == '~'
768
243 218
SELECT AuthID FROM PaperID_AuthID WHERE PaperID= '~' 
768
1229 219
SELECT COUNT(*) FROM ConfID_PaperID WHERE ConfID='~' ;
768
312 220
SELECT COUNT(PaperID) FROM PaperID_FieldID WHERE FieldID = '~' 
768
435 221
SELECT Summary from PaperID_Summary WHERE PaperID = '~' 
768
1580 222
SE

768
294 310
SELECT COUNT(AuthID) FROM PaperID_AuthID WHERE PaperID= '~' 
768
664 311
SELECT Affiliate FROM AffiliationID_Place_Affiliation WHERE Place LIKE '%Hertfordshire UK%'
768
1636 312
SELECT Topic FROM FieldID_Topic WHERE FieldID=~;
768
88 313
SELECT PaperID FROM PaperID_AuthID WHERE AuthID = '~'
768
41 314
SELECT PaperID FROM PaperID_AuthID WHERE AuthID = '~'
768
1612 315
SELECT ConfID FROM ConfID_Venue_Year WHERE Venue=Workshop On Analyzing Conversations In Text And Speech AND Year=Year;
768
131 316
SELECT COUNT(PaperID) FROM PaperID_AuthID WHERE AuthID = '~'
768
818 317
SELECT AuthID FROM AuthID_AffiliationID WHERE AffiliationID == '~'
768
707 318
SELECT Place,Affiliate,AffiliationID FROM AffiliationID_Place_Affiliation WHERE Affiliate LIKE '%Konstanz%'
768
810 319
SELECT AuthID FROM AuthID_AffiliationID WHERE AffiliationID == '~'
768
1544 320
SELECT COUNT(*) FROM ConfID_Venue_Year WHERE Year<2012;
768
1637 321
SELECT DISTINCT COUNT(Topic) FROM FieldID_Topic;
768
164 322
SELEC

768
1644 410
SELECT PaperID FROM KeywordID_PaperID WHERE KeywordID=~;
768
661 411
SELECT Affiliate FROM AffiliationID_Place_Affiliation WHERE Place LIKE '%Saabrucken Germany%'
768
1630 412
SELECT FieldID FROM FieldID_Topic WHERE Topic=Computational Linguitics on Twitter;
768
1545 413
SELECT COUNT(*) FROM ConfID_Venue_Year WHERE Year<2010;
768
376 414
SELECT FieldID FROM PaperID_FieldID WHERE PaperID = '~'
768
1230 415
SELECT COUNT(*) FROM ConfID_PaperID WHERE ConfID='~' ;
768
314 416
SELECT FieldID FROM PaperID_FieldID WHERE PaperID = '~'
768
442 417
SELECT Summary from PaperID_Summary WHERE PaperID = '~' 
768
917 418
SELECT AuthID,AffiliationID FROM AuthID_AffiliationID WHERE AuthID == '~'
768
319 419
SELECT FieldID FROM PaperID_FieldID WHERE PaperID = '~'
768
999 420
SELECT Name FROM AuthID_Name WHERE AuthID='~';
768
819 421
SELECT AuthID FROM AuthID_AffiliationID WHERE AffiliationID == '~'
768
983 422
SELECT Name FROM AuthID_Name WHERE AuthID='~';
768
1255 423
SELECT COUNT(*),ConfID

...

768
1562 838
SELECT COUNT(*) FROM ConfID_Venue_Year WHERE Year>1982;
768
689 839
SELECT Count(Affiliate) FROM AffiliationID_Place_Affiliation WHERE Place LIKE '%Stockholm%
768
851 840
SELECT Count(AuthID) FROM AuthID_AffiliationID WHERE AffiliationID == '~'
768
159 841
SELECT AuthID FROM PaperID_AuthID WHERE PaperID= '~' 
768
895 842
SELECT AuthID,AffiliationID FROM AuthID_AffiliationID WHERE AuthID == '~'
768
1318 843
SELECT PaperID FROM ConfID_PaperID WHERE ConfID='~
768
257 844
SELECT COUNT(AuthID) FROM PaperID_AuthID WHERE PaperID= '~' 
768
335 845
SELECT FieldID FROM PaperID_FieldID WHERE PaperID = '~'
768
1566 846
SELECT COUNT(*) FROM ConfID_Venue_Year WHERE Year>1993;
768
1039 847
SELECT AuthID FROM AuthID_Name WHERE Name='Francq, Pascal';
768
515 848
SELECT Title FROM PaperID_Title 
768
1015 849
SELECT Name FROM AuthID_Name WHERE AuthID='~';
768
768 850
SELECT Affiliate,Place FROM AffiliationID_Place_Affiliation WHERE AffiliationID == '~'
768
1461 851
SELECT COUNT(*) FROM ConfI

768
511 943
SELECT Summary from PaperID_Summary WHERE PaperID = '~' 
768
320 944
SELECT FieldID FROM PaperID_FieldID WHERE PaperID = '~'
768
19 945
SELECT PaperID FROM PaperID_AuthID WHERE AuthID = '~'
768
684 946
SELECT Count(Affiliate) FROM AffiliationID_Place_Affiliation WHERE Place LIKE '%Waxhaw NC%
768
35 947
SELECT PaperID FROM PaperID_AuthID WHERE AuthID = '~'
768
902 948
SELECT AuthID,AffiliationID FROM AuthID_AffiliationID WHERE AuthID == '~'
768
827 949
SELECT AuthID FROM AuthID_AffiliationID WHERE AffiliationID == '~'
768
1497 950
SELECT COUNT(*) FROM ConfID_Venue_Year WHERE Year=1994;
768
407 951
SELECT COUNT(FieldID) FROM PaperID_FieldID WHERE PaperID = '~'
768
920 952
SELECT AuthID,AffiliationID FROM AuthID_AffiliationID WHERE AuthID == '~'
768
537 953
SELECT Title FROM PaperID_Title WHERE PaperID = '~'
768
758 954
SELECT Affiliate,Place FROM AffiliationID_Place_Affiliation WHERE AffiliationID == '~'
768
1062 955
SELECT AuthID FROM AuthID_Name WHERE Name='BradenHarder, Li

768
207 1049
SELECT COUNT(AuthID) FROM PaperID_AuthID WHERE PaperID= '~' 
768
1577 1050
SELECT COUNT(*) FROM ConfID_Venue_Year WHERE Year>2014;
768
1122 1051
SELECT AuthID FROM AuthID_Name WHERE Name LIKE 'Lee, L%';
768
1463 1052
SELECT COUNT(*) FROM ConfID_Venue_Year WHERE Venue=TextGraphs Workshop On Graph Based Methods For Natural Language Processing;
768
8 1053
SELECT PaperID FROM PaperID_AuthID WHERE AuthID = '~'
768
1014 1054
SELECT Name FROM AuthID_Name WHERE AuthID='~';
768
1496 1055
SELECT COUNT(*) FROM ConfID_Venue_Year WHERE Year=1985;
768
1060 1056
SELECT AuthID FROM AuthID_Name WHERE Name='Moszczyxski, Radosxaw';
768
452 1057
SELECT Summary from PaperID_Summary WHERE PaperID = '~' 
768
253 1058
SELECT AuthID FROM PaperID_AuthID WHERE PaperID= '~' 
768
1327 1059
SELECT PaperID FROM ConfID_PaperID WHERE ConfID='~
768
896 1060
SELECT AuthID,AffiliationID FROM AuthID_AffiliationID WHERE AuthID == '~'
768
1599 1061
SELECT ConfID FROM ConfID_Venue_Year WHERE Venue=Workshop On Le

768
1354 1150
SELECT COUNT(*) FROM ConfID_PaperID WHERE PaperID LIKE 'W05-15%' ;
768
1604 1151
SELECT ConfID FROM ConfID_Venue_Year WHERE Venue=Proceedings of the 2009 Workshop on Text and Citation Analysis for Scholarly Digital Libraries (NLPIR4DL) AND Year=Year;
768
1539 1152
SELECT Venue FROM ConfID_Venue_Year WHERE ConfID=~;
768
795 1153
SELECT Place From AffiliationID_Place_Affiliation WHERE Affiliate LIKE 'Purdue University
768
645 1154
SELECT PaperID FROM PaperID_Title WHERE Title LIKE 'An Improved CRF based Chinese Langu%'
768
1012 1155
SELECT Name FROM AuthID_Name WHERE AuthID='~';
768
556 1156
SELECT Title FROM PaperID_Title WHERE PaperID = '~'
768
681 1157
SELECT Count(Affiliate) FROM AffiliationID_Place_Affiliation WHERE Place LIKE '%Leuven Belgium%
768
577 1158
SELECT PaperID FROM PaperID_Title WHERE Title = 'ETS: Discriminative Edit Models for Paraphrase Scoring'
768
1109 1159
SELECT AuthID FROM AuthID_Name WHERE Name LIKE 'Gar%';
768
1266 1160
SELECT ConfID FROM ConfID_P

768
1129 1252
SELECT AuthID FROM AuthID_Name WHERE Name LIKE 'Ro%';
768
565 1253
SELECT PaperID FROM PaperID_Title WHERE Title = 'Prediction of Maximal Projection for Semantic Role Labeling'
768
1413 1254
SELECT FieldID FROM ConfID_FieldID WHERE ConfID='~' ;
768
1021 1255
SELECT Name FROM AuthID_Name WHERE AuthID='~';
768
1025 1256
SELECT Name FROM AuthID_Name WHERE AuthID='~';
768
1411 1257
SELECT FieldID FROM ConfID_FieldID WHERE ConfID='~' ;
768
1585 1258
SELECT ConfID FROM ConfID_Venue_Year WHERE Venue=Workshop On Human Language Technology And Knowledge Management AND Year=Year;
768
1104 1259
SELECT AuthID FROM AuthID_Name WHERE Name LIKE 'Meng, Y%';
768
205 1260
SELECT COUNT(AuthID) FROM PaperID_AuthID WHERE PaperID= '~' 
768
34 1261
SELECT PaperID FROM PaperID_AuthID WHERE AuthID = '~'
768
775 1262
SELECT Affiliate,Place FROM AffiliationID_Place_Affiliation WHERE AffiliationID == '~'
768
508 1263
SELECT Summary from PaperID_Summary WHERE PaperID = '~' 
768
1451 1264
SELECT COUNT(

In [21]:
ele
temp3
y_train_repl

'SELECT COUNT(*) FROM ConfID_Venue_Year WHERE Venue=*SEM;'

In [5]:
#check almost match with the column names and fix the columns and table
#check all possiblity then revert back with reward for each answer
train_pre1={}
for i in range(768):
    train_pre1[str(i)]=[]
    for j in range(len(embed)):
        train_pre1[str(i)].append(embed[j][i])
train_pre={"embed":embed,"place_holder":place_holder,"match":match_all,"index":ind_all,"sql":y_train_rep}
#Get embedding and replace the values
# X_train.to_csv("X_train.csv",index=False)
# X_test.to_csv("X_test.csv",index=False)
# y_train.to_csv("Y_train.csv",index=False)
# y_test.to_csv("Y_test.csv",index=False)
t=pd.DataFrame(train_pre1)
t.to_csv("Train_embed.csv",index=False)

In [6]:
r=0
embed=[]
place_holder=[]
match_all=[]
for i in X_test.keys():
    if(X_test[i][-1]=='.' or X_test[i][-1]=='?'):
        arr=X_test[i][:-1].split(" ")
    else:
        arr=X_test[i].split(" ")
    temp=[]
    for j in range(len(arr)):
        if(arr[j] not in stop):
            temp.append(arr[j])
#     close=[]
#     print(i,X_train[i],temp)
    temp1=[]
    for val in temp:
        y=get_close_matches(val, [ele for ele in maping])
#         if(len(y)>0):
#             close.extend(y)
#             temp1.append("$col$")
#         else:
        temp1.append(val)
    match=[]
    temp2=[]
    for val in temp1:
        temp2.append(val.lower())
        for ele in maping:
            if(val in maping[ele]):
                match.append(val)
                temp2.pop()
                temp2.append("$val$")
#                 if(ele not in close):
#                     close.append(ele)
#     print(temp2) 
#     print(close,match) 
#     for val in table:
    r+=1
    gio=bc.encode([" ".join(temp2)])
    place_holder.append(temp2)
#     print(len(gio.flatten()))
    embed.append(gio.flatten())
    match_all.append(match)
#     if(r>10):
#         break
test_pre={"embed":embed,"place_holder":place_holder,"match":match_all}
test_pre1={}
for i in range(768):
    test_pre1[str(i)]=[]
    for j in range(len(embed)):
        test_pre1[str(i)].append(embed[j][i])
# train_pre={"embed":embed,"place_holder":place_holder,"match":match_all,"index":ind_all,"sql":y_train_rep}
#Get embedding and replace the values
# X_train.to_csv("X_train.csv",index=False)
# X_test.to_csv("X_test.csv",index=False)
# y_train.to_csv("Y_train.csv",index=False)
# y_test.to_csv("Y_test.csv",index=False)
t=pd.DataFrame(test_pre1)
t.to_csv("Test_embed.csv",index=False)

In [7]:
final=[]
for i in range(len(test_pre["embed"])):
#     print("1")
    val=test_pre["embed"][i]
    temp=0
    for j in range(len(train_pre['embed'])):
        com=train_pre["embed"][j]
        t=sum([val[k]*com[k] for k in range(len(val))])/(sum([val[k]**2 for k in range(len(val))])*sum([com[k]**2 for k in range(len(val))]))**(1/2)
        if(t>temp):
            temp=t
            max_val=j
    print(test_pre["place_holder"][i],temp,train_pre["place_holder"][max_val])
    sql=train_pre["sql"][max_val]
#     print(sql)
    for l in train_pre["index"][max_val]:
        if(l!=-1 and l<len(test_pre["match"][i])):
            sql=re.sub(r'~',test_pre["match"][i][l],sql)
    print(sql)
    final.append(sql)

['summary', 'paper', 'paperid', '$val$', ''] 1.0000000048163404 ['summary', 'paper', 'paperid', '$val$']
SELECT Summary from PaperID_Summary WHERE PaperID = 'W06-3121' 
['what', 'paper', 'id', 'paper', 'title', 'learning', 'transliteration', 'lexicons', 'from', 'the', 'web'] 0.9044833868181011 ['what', 'paper', 'id', 'paper', 'title', 'machine', 'translation', 'systems', 'and', 'computer', 'dictionaries', 'in', 'the', 'information', 'service:', 'ways', 'of', 'their', 'development', 'and', 'operation']
SELECT PaperID FROM PaperID_Title WHERE Title = 'Machine Translation Systems And Computer Dictionaries In The Information Service: Ways Of Their Development And Operation'
['what', 'paper', 'id', 'paper', 'title', 'improved', 'iterative', 'correction', 'distant', 'spelling', 'errors'] 0.9146028514398393 ['what', 'paper', 'id', 'paper', 'title', 'structured', 'ramp', 'loss', 'minimization', 'machine', 'translation']
SELECT PaperID FROM PaperID_Title WHERE Title = 'Structured Ramp Loss Mini

['list', 'papers', 'written', 'author', 'authorid', '$val$'] 0.999999984450869 ['list', 'papers', 'written', 'author', 'authorid', '$val$']
SELECT PaperID FROM PaperID_AuthID WHERE AuthID = 'A-17068'
['what', 'venue', 'conference', 'id', '$val$'] 0.9999999816456561 ['what', 'venue', 'conference', 'id', '$val$']
SELECT Venue FROM ConfID_Venue_Year WHERE ConfID=C-453;
['list', 'paper', 'ids', 'paper', 'title', 'starting', '$val$', 'software', 'environment', 'for', 'developing', 'nat'] 0.9426097061415096 ['list', 'paper', 'ids', 'paper', 'title', 'starting', '$val$', 'hedgehop']
SELECT PaperID FROM PaperID_Title WHERE Title LIKE 'A Hedgehop over a%'
['how', 'many', 'number', 'papers', 'published', 'id', 'starts', '$val$'] 0.956187781324538 ['how', 'many', 'number', 'conference', 'id', 'starts', '$val$']
SELECT COUNT(*) FROM ConfID_FieldID WHERE ConfID LIKE 'C-5%' ;
['print', 'summary', 'paper', '$val$'] 1.0000000024626146 ['print', 'summary', 'paper', '$val$']
SELECT Summary from PaperID_

['how', 'many', 'number', 'papers', 'published', 'conference', '$val$'] 0.9999999977380084 ['how', 'many', 'number', 'papers', 'published', 'conference', '$val$']
SELECT COUNT(*) FROM ConfID_PaperID WHERE ConfID='C-487' ;
['how', 'many', 'number', 'papers', 'published', 'conference', '$val$'] 0.9999999977380084 ['how', 'many', 'number', 'papers', 'published', 'conference', '$val$']
SELECT COUNT(*) FROM ConfID_PaperID WHERE ConfID='C-498' ;
['print', 'summary', 'paper', '$val$'] 1.0000000024626146 ['print', 'summary', 'paper', '$val$']
SELECT Summary from PaperID_Summary WHERE PaperID = 'D14-1121' 
['how', 'many', 'papers', 'written', '$val$'] 1.0000000018370332 ['how', 'many', 'papers', 'written', '$val$']
SELECT COUNT(PaperID) FROM PaperID_AuthID WHERE AuthID = 'A-3650'
['how', 'many', 'author', 'name', 'contains', 'roseacuten,', 'vict', ''] 0.9364111666029168 ['how', 'many', 'author', 'name', 'contains', 'audhk']
SELECT COUNT(*) FROM AuthID_Name WHERE Name LIKE '%Audhk%';
['list', 'a

['list', 'affliates', 'contain', 'open'] 0.9493678293773163 ['list', 'affliates', 'contain', 'computer']
SELECT Place,Affiliate,AffiliationID FROM AffiliationID_Place_Affiliation WHERE Affiliate LIKE '%Computer%'
['what', 'papers', 'written', '$val$'] 1.0000000145779167 ['what', 'papers', 'written', '$val$']
SELECT PaperID FROM PaperID_AuthID WHERE AuthID = 'A-12511'
['what', 'ids', 'authors', 'whose', 'name', 'begin', 'ish', ''] 0.9739521743894416 ['what', 'ids', 'authors', 'whose', 'name', 'begin', 'b']
SELECT AuthID FROM AuthID_Name WHERE Name LIKE 'B%';
['summary', 'paper', 'paperid', '$val$', ''] 1.0000000048163404 ['summary', 'paper', 'paperid', '$val$']
SELECT Summary from PaperID_Summary WHERE PaperID = 'W97-0310' 
['where', 'affiliate', 'ge', 'aerospace', 'loacted'] 0.9509958420029385 ['where', 'affiliate', 'ibm', 'corp', 'loacted']
SELECT Place From AffiliationID_Place_Affiliation WHERE Affiliate LIKE 'IBM Corp
['how', 'many', 'author', 'name', 'contains', 'kovaleva,', '$val$

['which', 'papers', 'keyword', 'id', '$val$'] 0.9999999852332186 ['which', 'papers', 'keyword', 'id', '$val$']
SELECT PaperID FROM KeywordID_PaperID WHERE KeywordID=K-756;
['list', 'authids', '$val$'] 0.9999999831612784 ['list', 'authids', '$val$']
SELECT AuthID FROM AuthID_AffiliationID WHERE AffiliationID == 'AF-673'
['what', 'papers', 'written', '$val$'] 1.0000000145779167 ['what', 'papers', 'written', '$val$']
SELECT PaperID FROM PaperID_AuthID WHERE AuthID = 'A-1845'
['how', 'many', 'papers', 'written', '$val$'] 1.0000000018370332 ['how', 'many', 'papers', 'written', '$val$']
SELECT COUNT(PaperID) FROM PaperID_AuthID WHERE AuthID = 'A-6140'
['what', 'number', 'authors', '$val$'] 1.0000000134440967 ['what', 'number', 'authors', '$val$']
SELECT COUNT(AuthID) FROM PaperID_AuthID WHERE PaperID= 'P98-1042' 
['print', 'summary', 'paper', '$val$'] 1.0000000024626146 ['print', 'summary', 'paper', '$val$']
SELECT Summary from PaperID_Summary WHERE PaperID = 'W08-1136' 
['list', 'field', 'i

['how', 'many', 'number', 'papers', 'published', 'conference', '$val$'] 0.9999999977380084 ['how', 'many', 'number', 'papers', 'published', 'conference', '$val$']
SELECT COUNT(*) FROM ConfID_PaperID WHERE ConfID='C-431' ;
['how', 'many', 'author', 'name', 'contains', 'ono,', 'takah', ''] 0.936264946326394 ['how', 'many', 'author', 'name', 'contains', 'he,', 'jin']
SELECT COUNT(*) FROM AuthID_Name WHERE Name LIKE '%He, Jin%';
['what', 'title', 'paper', 'paperid', '$val$'] 1.0000000010291723 ['what', 'title', 'paper', 'paperid', '$val$']
SELECT Title FROM PaperID_Title WHERE PaperID = 'C94-2129'
['name', 'author', 'whose', 'id', '$val$'] 1.0000000031666008 ['name', 'author', 'whose', 'id', '$val$']
SELECT Name FROM AuthID_Name WHERE AuthID='A-12848';
['what', 'paper', 'id', 'paper', 'title', 'an', 'algorithm', 'unsupervised', 'transliteration', 'mining', 'application', 'word', 'alignment'] 0.9352117001367178 ['what', 'paper', 'id', 'paper', 'title', 'structured', 'ramp', 'loss', 'minimiz

['what', 'ids', 'authors', 'whose', 'name', 'begin', 'moncecchi,', 'gui', ''] 0.9694983444877489 ['what', 'ids', 'authors', 'whose', 'name', 'begin', 'picanco,', 'ge']
SELECT AuthID FROM AuthID_Name WHERE Name LIKE 'Picanco, Ge%';
['what', 'fields', 'associated', 'paperid', '$val$'] 0.9999999825516308 ['what', 'fields', 'associated', 'paperid', '$val$']
SELECT FieldID FROM PaperID_FieldID WHERE PaperID = 'P03-1044'
['count', 'numberfields', 'associated', 'paperid', '$val$'] 0.9999999908745282 ['count', 'numberfields', 'associated', 'paperid', '$val$']
SELECT COUNT(FieldID) FROM PaperID_FieldID WHERE PaperID = 'W98-1217'
['how', 'many', 'total', 'conferences', 'held', 'proceedings', '$val$', '$val$', '2010', 'workshop', 'computational', 'linguistics', 'world', 'social', 'media'] 0.9790374208303304 ['how', 'many', 'total', 'conferences', 'held', 'proceedings', '$val$', '$val$', '2010', 'second', 'workshop', 'computational', 'approaches', 'linguistic', 'creativity']
SELECT COUNT(*) FROM C

['where', 'affiliate', 'university', 'umea', 'loacted'] 0.9427059440980466 ['where', 'affiliate', 'university', 'chicago', 'loacted']
SELECT Place From AffiliationID_Place_Affiliation WHERE Affiliate LIKE 'University of Chicago
['how', 'many', 'conferences', 'held', '1973'] 0.9999999814830633 ['how', 'many', 'conferences', 'held', '1973']
SELECT COUNT(*) FROM ConfID_Venue_Year WHERE Year>1973;
['list', 'authids', '$val$'] 0.9999999831612784 ['list', 'authids', '$val$']
SELECT AuthID FROM AuthID_AffiliationID WHERE AffiliationID == 'AF-1123'
['list', 'field', 'ids', 'associated', 'paperid', '$val$'] 1.0000000075266713 ['list', 'field', 'ids', 'associated', 'paperid', '$val$']
SELECT FieldID FROM PaperID_FieldID WHERE PaperID = 'C65-1006'
['how', 'many', 'total', 'conferences', 'held', 'workshop', 'on', 'the', 'acquisition', 'of', 'lexical', 'knowledge', 'from', 'text'] 0.9464514929357498 ['how', 'many', 'total', 'conferences', 'held', 'siglex', 'workshop', 'on', 'deep', 'lexical', 'acqu

['how', 'many', 'number', 'conference', 'conducted', 'field', '$val$'] 1.000000003175937 ['how', 'many', 'number', 'conference', 'conducted', 'field', '$val$']
SELECT COUNT(*) FROM ConfID_FieldID WHERE FieldID='F-2' ;
['list', 'paper', 'id', 'paper', 'conference', 'id', '$val$'] 0.9999999981192202 ['list', 'paper', 'id', 'paper', 'conference', 'id', '$val$']
SELECT PaperID FROM ConfID_PaperID WHERE ConfID='C-452
['what', 'title', 'paper', 'paperid', '$val$'] 1.0000000010291723 ['what', 'title', 'paper', 'paperid', '$val$']
SELECT Title FROM PaperID_Title WHERE PaperID = 'W10-4236'
['summary', 'paper', 'paperid', '$val$', ''] 1.0000000048163404 ['summary', 'paper', 'paperid', '$val$']
SELECT Summary from PaperID_Summary WHERE PaperID = 'W09-3926' 
['count', 'numberfields', 'associated', 'paperid', '$val$'] 0.9999999908745282 ['count', 'numberfields', 'associated', 'paperid', '$val$']
SELECT COUNT(FieldID) FROM PaperID_FieldID WHERE PaperID = 'X96-1021'
['what', 'ids', 'authors', 'whose'

In [10]:
rt={"Sql":final,"Comp":y_test}
rt=pd.DataFrame(rt)
data_final=rt.to_csv("Result.csv")

In [16]:
count=0
y_t=[]
for j in y_test.keys():
    y_t.append(y_test[j])
for i in range(len(final)):
    if(final[i].strip().lower()==y_t[i].strip().lower()):
        count+=1
print(count/len(final))

0.6447761194029851
